# LIBRARY

In [1]:
#General
import numpy as np    
import pandas as pd

#Keras General
from keras.models import Sequential    #initialize NN
from keras.utils import to_categorical
from keras.callbacks import Callback   #save model
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint #choose best model
from keras import backend as K              #create new metrics
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import load_model

#Keras layers
from keras.layers.core import Dense, Activation 
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D,AveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers.normalization import BatchNormalization #normalization

#other technique
from keras import optimizers         #optimizer
from keras.optimizers import Adam    #Adam optimizer



Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\py37\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\ProgramData\Anaconda3\envs\py37\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\ProgramData\Anaconda3\envs\py37\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\ProgramData\Anaconda3\envs\py37\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\ProgramData\Anaconda3\envs\py37\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: The specified module could not be found.


Failed to load the native TensorFlow runtime.



# METHODOLOGY

In [8]:
"""

There are only 64 picture of football ball. 
Problem is a simple classification with Deep Learning.

For this are required two classes:
- Class 0: Football ball
- Class 1: General

For class 1, I used 468 random picture take from google.
Instead for class 0, I downloaded other 171 football ball pictures 
from Google Image. (I also used pictures not of football ball but
of things with drawn football ball, for example pillow.)

So the dataset is with 539 pictures where 171 are class 0 and 
468 are class 1.
Where 60% will be training and 40% testing.

Because them are few, for increasing the size of dataset I used data
augmentation.

The architecture of Deep learning is a CNN with 3 Layer of Convolutional
layer with maxpooling and a ANN.

Images have different size, so I resized their to width and height 100x100.
Each picture is loaded on a tensor 100x100x3, because their are colored
and I used three channel ('rgb'). Can see this as 539x3 matrixs with
100 rows and 100 columns where each coordinate (x,y) indicate a pixel
with value from 0 to 255 for each channel.
Input will be (before data augmentantion) of 539x100x100x3.

This is big! The role of CNN is to reduce the images without 
losing information.

I chose to use a batch size of 32 samples for propagated through 
the network. This mean take the first 32 samples and train a CNN and
will update the weights, the will be take the following 32 samples and
will train again the CNN updating again the weights. 


Instead learning rate is a hyper-parameter that control how much are adjusting 
weights.

To extract features of a image, my CNN uses a kernel 3x3 with
no stride (we can image it as a little matrix 3x3 that move on each image). 
They use different filters on each ConvLayer and take the most important
feature with Maxpooling (2x2). So a filter is a little matrix 3x3 with usually
value -1,0,1... that will be multiply with a part of the image and will be
take for last the biggest value. This for each filter.
Will apply a relu function.


The Fully-Connected layer with Flatten is learning a possibly 
non-linear function in that space. The model is able
to see the dominating features in image and classify them
using the Softmax Classification.

I worked on CPU with just 10 epoches with optimizer Adam and
with loss function "categorical_crossentropy".

I obtained an accuracy of more 0.9.


""" 

'\n\nWe have just 64 football images. Problem is a Classification.\nSo I need of two classes:\n- Class 0: Football\n- Class 1: General\n\nSo I insert 468 random picture from google \n(101_ObjectCategories/BACKGROUD_GOOGLE).\nThen I download others 171 football pictures from Google Image.\n\nSo I had an dataset of 539 pictures wi\n\n'

# DATA

In [12]:
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

import tensorflow as tf

num_filters = 16
filter_size = 3
pool_size = 2
larg=100
lung=100
ch=3
f_1='relu'
f_2='softmax'


model=Sequential()
model.add(Conv2D(num_filters*4, filter_size, input_shape=(width, height, ch),activation=f_1))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(num_filters*2, filter_size,activation=f_1))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(num_filters, filter_size,activation=f_1))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(layers.Dropout(0.60))
model.add(Flatten())
model.add(Dense(10, activation=f_1))
model.add(Dense(2, activation=f_2))

#applichiamo delle traformazioni casuacategorical_crossentropyli per aumentare il numero di esempi nel training
train_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)



training_set = train_datagen.flow_from_directory(r'...\training',
                                            target_size = (width, height),
                                            batch_size = n_batch,
                                            color_mode='rgb',
                                            classes = ['Soccer','Google'],
                                           class_mode = 'categorical',
                                           shuffle=True)


test_set = test_datagen.flow_from_directory(r'...\testing',
                                            target_size = (width, height),
                                            batch_size = n_batch,
                                            color_mode='rgb',
                                            classes = ['Soccer','Google'],
                                           class_mode = 'categorical',
                                           shuffle=True)


optimizer = Adam(lr=0.002)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

#early_stop = EarlyStopping(monitor='loss', min_delta=0.001, patience=3, mode='min', verbose=1)
checkpoint = ModelCheckpoint('model_best_weights.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', period=1)


model.fit_generator(training_set,
                         steps_per_epoch = train_dim,   #numero di immagini nel trainig set (tutte le osservazioni vengono passate durante ogni epoca)
                         epochs = n_epoches,
                         validation_data = test_set,
                         validation_steps = test_dim,verbose=1,callbacks = [checkpoint])  

Found 450 images belonging to 2 classes.
Found 188 images belonging to 2 classes.
Epoch 1/10
7000/7000 [==============================] - 2281s 326ms/step - loss: 0.0419 - acc: 0.9841 - val_loss: 0.5524 - val_acc: 0.8777

Epoch 00001: val_loss improved from inf to 0.55243, saving model to model_best_weights.h5
Epoch 2/10
7000/7000 [==============================] - 2257s 322ms/step - loss: 0.0072 - acc: 0.9983 - val_loss: 0.5239 - val_acc: 0.9255

Epoch 00002: val_loss improved from 0.55243 to 0.52394, saving model to model_best_weights.h5
Epoch 3/10
7000/7000 [==============================] - 2265s 324ms/step - loss: 0.0042 - acc: 0.9990 - val_loss: 0.5897 - val_acc: 0.9043

Epoch 00003: val_loss did not improve from 0.52394
Epoch 4/10
7000/7000 [==============================] - 2271s 324ms/step - loss: 0.0069 - acc: 0.9987 - val_loss: 0.5887 - val_acc: 0.9149

Epoch 00004: val_loss did not improve from 0.52394
Epoch 5/10
7000/7000 [==============================] - 30654s 4s/step -

In [92]:
#evaluate on original images

model=load_model("model_best_weights.h5")
print("Loaded model from disk")

x=range(1,65)    # there are 64 images in folder
L=[]             # save image names in this list
for k in x:
    L.append('image_'+str(k).zfill(4)+'.jpg') #use lpad to generate name

pred=0
n=0
for li in L:
    fold='SoccerBall'
    img=li
    test_image = image.load_img('.../'+img, target_size = (larg, lung),grayscale=False)

    test_image = image.img_to_array(test_image)

    test_image = np.expand_dims(test_image, axis = 0)/255
    if model.predict(test_image)[0][0]>0.5:
        pred+=1

    
tp=pred/len(L)
tp



0.90625

# PROVA RETE

In [ ]:
from keras.models import load_model
model=load_model("model_best_weights.h5")
print("Loaded model from disk")
